In [82]:
"""
from Position import Position
from Move import Move
from InvalidMoveException import *


from Game import Game
"""
from Rock import Rock
import unittest


In [83]:
from Pile import Pile
from Player import Player
from Position import Position
from Move import Move
from InvalidMoveException import *

class Game:
    """
    The Game class represents a game of Gobblet Gobblers. and is resoponsible for
    * Maintaining state of pieces on board and in reserves
    * Executing player moves while enforcing rules
    * Validating move legality
    * Checking win conditions
    * Providing available moves for current player

    Attributes:
        player (list[Player]): A list of the two players in the game.
        grid (list[list[Pile]]): A 4x4 grid representing the game board.
        move_history (list[Move]): A list of all moves made in the game.
        possible_moves (list[Move]): A list of all possible moves for the current player.

    Methods:
        print_grid(): Prints the current state of the game board.
        is_valid(move: Move): Checks if a move is valid.
        do_turn(move: Move): Executes a move.
        check_win(): Checks if the current state of the game is a win.
        has_legalMoves(): Checks if any legal move is available for the current player.
        check_three_repetitions(): Checks if the current state of the game has three cycles of repeated moves.
        check_tie(): Checks if the current state of the game is a tie.
        generate_possible_moves(player_id): Generates a list of all possible moves for a player.
    """
  
    player: list[Player]
    grid: list[list[Pile]]
    move_history: list[Move]
    possible_moves: list[Move]
    

    
    def __init__(self, player1_name, player2_name) -> None:
        """
            Initializes a new instance of the Game class.
            Args:
            player1_name (str): The name of the first player.
            player2_name (str): The name of the second player.
        """
        self.player = [Player(player1_name, 0), Player(player2_name, 1)]
        self.grid = [
            [Pile(),Pile(),Pile(),Pile()],
            [Pile(),Pile(),Pile(),Pile()],
            [Pile(),Pile(),Pile(),Pile()],
            [Pile(),Pile(),Pile(),Pile()]
            ]
        self.move_history = []  # Changed from self.game_history = []
        self.possible_moves = []

    
    def print_grid(self) -> None:
      """
      Prints the current state of the game board.
      """
      for row in self.grid:
        for cell in row:
          print(cell.rocks[-1].size if cell.rocks else '#', end=' ')
        print("\n")
      print("\n")

    
    def is_valid(self, move: Move) -> bool:
        """
            Checks if the move is valid.
            Args:
                move (Move): The move to check.
        """

        if move.from_grid and self.grid[move.from_grid.x][move.from_grid.y].rocks and self.grid[move.from_grid.x][move.from_grid.y].rocks[-1].id != move.player_id:
            raise MoveFromAnotherPlayerException("You cannot play from another player's rocks.")

        if move.from_grid and not self.grid[move.from_grid.x][move.from_grid.y].rocks:
            raise MoveFromEmptyGridException("You cannot play from an empty cell.")
        
        if move.from_grid and move.from_grid.x == move.to_grid.x and move.from_grid.y == move.to_grid.y:
            raise MoveToSamePositionException("You cannot play on the same cell.")
        
        if move.from_grid and self.grid[move.to_grid.x][move.to_grid.y].rocks and self.grid[move.from_grid.x][move.from_grid.y].rocks[-1].size <= self.grid[move.to_grid.x][move.to_grid.y].rocks[-1].size:
            raise MoveToSmallerRockException("You cannot play a smaller rock on top of a larger one.")
        
        if move.from_pile!= None and self.player[move.player_id].piles[move.from_pile].rocks and self.grid[move.to_grid.x][move.to_grid.y].rocks and self.player[move.player_id].piles[move.from_pile].rocks[-1].size <= self.grid[move.to_grid.x][move.to_grid.y].rocks[-1].size:
            raise MoveToSmallerRockException("You cannot play a smaller rock on top of a larger one.")
       

        if self.grid[move.to_grid.x][move.to_grid.y].rocks and self.grid[move.to_grid.x][move.to_grid.y].rocks[-1].id != move.player_id and not self.is_able_to_win(move.to_grid):
            raise MoveWithNo3RocksException("You cannot play on another player's rock unless they have 3 in a row")
        return True


    def do_turn(self, move: Move) -> None:
        """
            Executes the given move.
            Args:
                move (Move): The move to execute.
        """
        #TODO if you are using true or false instead of exceptions, you can use the following code to catch errors
       
        self.is_valid(move)
         
        self.move_history.append(move)
        if move.from_pile != None:
          self.grid[move.to_grid.x][move.to_grid.y].push(self.player[move.player_id].piles[move.from_pile].pop())
        elif move.from_grid:
          self.grid[move.to_grid.x][move.to_grid.y].push(self.grid[move.from_grid.x][move.from_grid.y].pop())
        
        # Reset the possible moves list after a move has been made
        self.possible_moves = None

    def check_win(self):
        """
        Checks if the current state of the game if it is a win
        Returns:
            The ID of the player who won the game if there is a winner, None otherwise
        """
        cnt = 0
        for i in range(4):
            for j in range(4):
                if self.grid[i][j].rocks and self.grid[0][j].rocks and self.grid[i][j].rocks[-1].id == self.grid[0][j].rocks[-1].id:
                    cnt += 1
                if cnt == 4:
                    return self.grid[0][j].rocks[-1].id
        cnt = 0
        for i in range(4):
              for j in range(4):
                  if self.grid[j][i].rocks and self.grid[0][j].rocks and self.grid[j][i].rocks[-1].id == self.grid[0][j].rocks[-1].id:
                      cnt += 1
                  if cnt == 4:
                      return self.grid[i][0].rocks[-1].id

        if all(self.grid[i][i].rocks and self.grid[i][i].rocks[-1].id == self.grid[0][0].rocks[-1].id for i in range(4)):
            return self.grid[0][0].rocks[-1].id

        # Check anti-diagonal
        if all(self.grid[i][3 - i].rocks and self.grid[i][3 - i].rocks[-1].id == self.grid[0][3].rocks[-1].id for i in range(4)):
            return self.grid[0][3].rocks[-1].id


    def has_legalMoves(self, player_id):
        """
        Checks if any legal move is available for the current player.
        Returns:
            True if a legal move is available, False otherwise.
        """ 

        if self.possible_moves is None:
            self.possible_moves = self.generate_possible_moves(player_id)
        if len(self.possible_moves) > 0:
            return True
        return False


    def check_three_repetitions(self):
        """
        Checks if the current state of the game has three cycles of repeated moves.

        Returns:
            True if three repetitions are found, False otherwise.
        """
        # Check if it's still early in the game
        if len(self.move_history) < 6:
            return False
        # Check for three consecutive identical entries in the history
        if self.move_history[-1] == self.move_history[-4] and self.move_history[-2] == self.move_history[-5] and self.move_history[-3] == self.move_history[-6]:
            return True
        return False


    def check_tie(self): 
        """
          Checks if the current state of the game has three repetitions of identical moves.
          Returns:
              True if three repetitions are found, False otherwise.
        """
        #it's been three cycling moves with no winner or there are no legal moves left
        if self.check_three_repetitions() or (not self.has_legalMoves()):
            return True
        return False

    def generate_possible_moves(self, player_id) -> list[Move]:
        """
        Generates a list of all possible moves for the given player.
        Args:
            player_id (int): The ID of the player to generate moves for.
        Returns:
            A list of all possible moves for the given player.
        """
        # Check if the list of possible moves has already been generated
        if self.possible_moves:
            return self.possible_moves
        
        # Initialize possible_moves as an empty list
        self.possible_moves = []

        #check all combinations of moves to grid
        for to_grid_x in range(4):
            for to_grid_y in range(4):

                #check all combinations of moves from piles
                for from_pile_index in range(3):
                    # Check if playing from a pile is legal
                    try:
                        move = Move(player_id, Position(to_grid_x, to_grid_y), None, from_pile=from_pile_index)
                        self.is_valid(move)
                        self.possible_moves.append(move)
                    except:
                    #if not pass
                        pass
                #check all combinations of moves from grid
                for from_grid_x in range(4):
                    for from_grid_y in range(4):
                        # Check if moving a rock from the grid is legal
                        try:
                            move = Move(player_id, Position(to_grid_x, to_grid_y), Position(from_grid_x, from_grid_y), None)
                            self.is_valid(move)
                            self.possible_moves.append(move)
                        except:
                            pass
                        
                            
        return self.possible_moves


# Tests

test for is_valid()

In [84]:
"""
    Tests for the is valid function.
"""
class TestIsValid(unittest.TestCase):

    def setUp(self):
        self.game = Game("Player1", "Player2")
    
    def tearDown(self):
        del self.game

    
    """
        Tests for the is_valid method.
    """
    def test_move_from_both_grid_and_pile(self):
        with self.assertRaises(MoveFromBothGridAndPileException):
            move = Move(0, Position(0, 0), Position(1, 1), 0)
            self.game.is_valid(move)

    def test_move_from_neither_grid_nor_pile(self):
        with self.assertRaises(MoveFromNeitherGridNorPileException):
            move = Move(0, Position(0, 0), None, None)
            self.game.is_valid(move)

    def test_move_from_another_player(self):
        self.game.grid[0][0].push(Rock(1, 1))
        with self.assertRaises(MoveFromAnotherPlayerException):
            move = Move(0, Position(0, 0), Position(0, 0))
            self.game.is_valid(move)

    def test_move_from_empty_grid(self):
        with self.assertRaises(MoveFromEmptyGridException):
            move = Move(0, Position(0, 0), Position(1, 1))
            self.game.is_valid(move)

    def test_move_to_smaller_rock(self):
        self.game.grid[0][0].push(Rock(1, 0))
        self.game.grid[1][1].push(Rock(2, 1))
        with self.assertRaises(MoveToSmallerRockException):
            move = Move(0, Position(1, 1), Position(0, 0))
            self.game.is_valid(move)

    def test_move_to_same_position(self):
        self.game.grid[0][0].push(Rock(1, 0))
        with self.assertRaises(MoveToSamePositionException):
            move = Move(0, Position(0, 0), Position(0, 0))
            self.game.is_valid(move)

    def test_move_with_no_3_rocks(self):
        self.game.grid[0][0].push(Rock(1, 1))
        with self.assertRaises(MoveWithNo3RocksException):
            move = Move(0, Position(0, 0), None, 0)
            self.game.is_valid(move)
    
    def test_move_to_none(self):
        with self.assertRaises(MoveToNonePositionException):
            move = Move(0, None, None)
            self.game.is_valid(move)
            
    def test_valid_move(self):
        move = Move(0, Position(0, 0), None, 0)
        self.assertTrue(self.game.is_valid(move))

test for do_turn()

In [85]:
class TestDoTurn(unittest.TestCase):
    def setUp(self):
        self.game = Game("Player1", "Player2")
        
    def test_do_turn_from_pile(self):
        # Test a valid move
        move = Move(0, Position(0, 0), None, 0)
        self.game.do_turn(move)

        # Assert that the move was executed correctly
        assert self.game.grid[0][0].rocks[-1].id == move.player_id

    def test_do_turn_invalid_move(self):
        # Test an invalid move

        with self.assertRaises(Exception):
            move = Move(1, Position(0, 0), Position(1, 1), 0)
            self.game.do_turn(move)

    def test_do_turn_from_grid(self):
        # Test a move from the grid
        move = Move(0, Position(0, 0), Position(1, 1), None)

        self.game.grid[1][1].push(Rock(1, 0))
        self.game.do_turn(move)

        # Assert that the move was executed correctly
        assert self.game.grid[0][0].rocks[-1].id == move.player_id

Test for check_win()

In [86]:


class TestCheckWin(unittest.TestCase):
    def setUp(self):
        self.game = Game("Player1", "Player2")

    def test_no_winner(self):
        self.assertIsNone(self.game.check_win())

    def test_player1_wins_row(self):
        for i in range(4):
            self.game.grid[0][i].push(Rock(1, 0))
        self.assertEqual(self.game.check_win(), 0)

    def test_player2_wins_column(self):
        for i in range(4):
            self.game.grid[i][0].push(Rock(1, 1))
        self.assertEqual(self.game.check_win(), 1)

    def test_player1_wins_diagonal(self):
        for i in range(4):
            self.game.grid[i][i].push(Rock(1, 0))
        self.assertEqual(self.game.check_win(), 0)

    def test_player2_wins_reverse_diagonal(self):
        for i in range(4):
            self.game.grid[i][3-i].push(Rock(1, 1))
        self.assertEqual(self.game.check_win(), 1)
    
    def test_variable_size(self):
        for i in range(4):
            self.game.grid[0][i].push(Rock(i+1, 0))
        self.assertEqual(self.game.check_win(), 0)
    
    
    


 

##    def test_check_three_repetitions(self):
        # Test the check_three_repetitions method
        # Add your test cases here
##        pass

##    def test_check_tie(self):
        # Test the check_tie method
        # Add your test cases here
##        pass



test check_tie()

test generate_possible_moves()

In [87]:
class TestGame(unittest.TestCase):
    def setUp(self):
        self.game = Game("Player 1", "Player 2")  # Initialize a new game

    def test_generate_possible_moves(self):
        possible_moves = self.game.generate_possible_moves(0)
        self.assertEqual(len(possible_moves), 48)

    def test_no_possible_moves(self):
        self.game = Game("Player 1", "Player 2")
        # Fill the grid with larger rocks on top of smaller ones
        for i in range(4):
            for j in range(4):
                self.game.grid[i][j].push(Rock(2, 0))
                self.game.grid[i][j].push(Rock(3, 1))
        # Empty the piles
        for pile in self.game.player[0].piles:
            while pile.rocks:
                pile.pop()
        for pile in self.game.player[1].piles:
            while pile.rocks:
                pile.pop()
        possible_moves = self.game.generate_possible_moves(0)
        self.assertEqual(len(possible_moves), 0)

test for has_legalMoves()

In [88]:
class TestGame(unittest.TestCase):
    def setUp(self):
        self.game = Game("Player 1", "Player 2")  # Initialize a new game

    def test_has_legalMoves_when_possible_moves_is_None(self):
        self.game.possible_moves = None
        self.assertTrue(self.game.has_legalMoves(0))

    def test_has_legalMoves_when_possible_moves_is_empty(self):
        self.game.possible_moves = []
        self.assertFalse(self.game.has_legalMoves(0))

    def test_has_legalMoves_when_possible_moves_is_not_empty(self):
        self.game.possible_moves = [Move(0, Position(0, 0), None, 0)]  # Assume this is a valid move
        self.assertTrue(self.game.has_legalMoves(0))
    
    #integration tests
        
    def test_has_legalMoves_integrated_with_generate_possible_moves(self):
       # Generate possible moves for the current player
       self.game.possible_moves = self.game.generate_possible_moves(0)
       self.assertTrue(self.game.has_legalMoves(0))  # There should be possible moves, so it should return True

    def test_has_legalMoves_integrated_with_generate_possible_moves_when_no_moves(self):
         # Fill the grid with larger rocks on top of smaller ones
         for i in range(4):
              for j in range(4):
                self.game.grid[i][j].push(Rock(2, 0))
                self.game.grid[i][j].push(Rock(3, 1))
         # Empty the piles
         for pile in self.game.player[0].piles:
              while pile.rocks:
                pile.pop()
         for pile in self.game.player[1].piles:
              while pile.rocks:
                pile.pop()
         # Generate possible moves for the current player
         self.game.possible_moves = self.game.generate_possible_moves(0)
         self.assertFalse(self.game.has_legalMoves(0))
        
    def test_has_legalMoves_integrated_with_do_turn(self):
        # Execute a move
        self.game.do_turn(Move(0, Position(0, 0), None, 0))
        # Check if there are any possible moves left
        self.assertTrue(self.game.has_legalMoves(0))

In [89]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

......................E.
ERROR: test_move_with_no_3_rocks (__main__.TestIsValid.test_move_with_no_3_rocks)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Sara\AppData\Local\Temp\ipykernel_34792\2855008839.py", line 54, in test_move_with_no_3_rocks
    self.game.is_valid(move)
  File "C:\Users\Sara\AppData\Local\Temp\ipykernel_34792\1950001913.py", line 92, in is_valid
    if self.grid[move.to_grid.x][move.to_grid.y].rocks and self.grid[move.to_grid.x][move.to_grid.y].rocks[-1].id != move.player_id and not self.is_able_to_win(move.to_grid):
                                                                                                                                            ^^^^^^^^^^^^^^^^^^^
AttributeError: 'Game' object has no attribute 'is_able_to_win'

----------------------------------------------------------------------
Ran 24 tests in 0.051s

FAILED (errors=1)
